In [5]:
### First we handle our files ###
tree_infile = f"../000_000_341.tree"
alignment_infile = f"../000_000_341.fas.alg"
table_infile = f"../000_000_341.tsv"
swissprot_infile = f"../uniprot_2018_09.json"

### And we store them in different variables ###

try:
    with open(table_infile, "r") as table_file:
        table_info = table_file.readlines()
except:
    print ("No table infile.")
        

try:
    with open(alignment_infile, "r") as alignment_file:
        alignment_info = alignment_file.read()
except:     
    print ("No alignment infile.")
        

import json
try:
    with open(swissprot_infile, "r") as swissprot_file:
        swissprot_info = [json.loads(line) for line in swissprot_file]
except:
    print ("No swissprot infile.")

In [ ]:
## ESTO ES UNA CHULETA PARA BORRAR ###
swissprot_info = json.loads(open(swissprot_infile).read())



    json_data = [json.loads(line) for line in handle]

for i, line in enumerate(f):
    try:
        d = json.loads(line)
    except json.decoder.JSONDecodeError:
        print('Error on line', i + 1, ':\n', repr(line))

In [78]:
swiss_hit_hash = {}
for line in table_info:
    hit_type = line.split("\t")[2]
    hit_name = line.split("\t")[1]
    if (hit_type == ("swissprot_best" or "swissprot_exact") and hit_name not in swiss_hit_hash):
        # Aqui ponerle que busque esa entrada en el archivo JSON y guardar las anotaciones como el valor del hash
        swiss_hit_hash[hit_name] = "test"
        print (swiss_hit_hash)


{'Q88SK6': 'test'}
{'Q88SK6': 'test', 'Q9BQ69': 'test'}
{'Q88SK6': 'test', 'Q9BQ69': 'test', 'Q8PHB6': 'test'}
{'Q88SK6': 'test', 'Q9BQ69': 'test', 'Q8PHB6': 'test', 'Q44020': 'test'}
{'Q88SK6': 'test', 'Q9BQ69': 'test', 'Q8PHB6': 'test', 'Q44020': 'test', 'O67077': 'test'}


In [45]:
### ESTO ES CHULETA PARA BORRAR ###
table_info[line] = [line.split("\t") for line in table_info]
print (table_info[0])


[i.split('\t', 1)[0] for i in l]

0
